# 生成双比特 待重构算符 和 生成随机初、末状态数据集

In [9]:
from mindquantum import *
import numpy as np
np.random.seed(1)

qubit_num = 2 # 待重构算符的比特数
data_num = 2000 # 用于训练的样本总数
u_num = 10 # 要构建的算符数

# 搭建用于产生量子初态的线路
sc = U3('00','01','02', 0)
sc += U3('10','11','12', 1)
sc += X.on(0,1)
sc += RZ('03').on(0)
sc += RY('13').on(1)
sc += X.on(1,0)
sc += RY('14').on(1)
sc += X.on(0,1)
sc += U3('05','06','07', 0)
sc += U3('15','16','17', 1)

params_name = sc.params_name

# 生成随机初态数据
state_list = []
for i in range(data_num):
    params = 2*np.pi*np.random.rand(len(params_name))
    state = sc.get_qs(pr=dict(zip(params_name, params)), ket=False)
    state_list.append(state)
        
init_states = np.array(state_list)
print('\n初态数据维度为：', init_states.shape)

u_list = []

for i in range(u_num):
    params = 2*np.pi*np.random.rand(len(params_name))
    u_mat = sc.matrix(dict(zip(sc.params_name, params)))
    u_list.append(u_mat)

u_mats = np.array(u_list)
print('算符数据维度为', u_mats.shape)

# 根据初态，对每个幺正算符产生末态
state_list = []
for u_mat in u_list:
    state_list.append([u_mat@init_state for init_state in init_states])
    
final_states = np.array(state_list)
print('\n末态数据维度为：', final_states.shape)

# 拆分数据集
train_x = init_states[:int(data_num*0.7)] # 按 7:3 分割
eval_x = init_states[int(data_num*0.7):]
train_y = final_states[:, :int(data_num*0.7)]
eval_y = final_states[:, int(data_num*0.7):]

# 分别保存 新训练集 和 新验证集
np.save('./src/2_qubit_train_x.npy', train_x)
np.save('./src/2_qubit_eval_x.npy', eval_x)
np.save('./src/2_qubit_train_y.npy', train_y)
np.save('./src/2_qubit_eval_y.npy', eval_y)
np.save('./src/2_qubit_u.npy', u_mats)
print('导出完成啦')

# 再次导入，以验证是否完成任务
a = np.load('./src/2_qubit_train_x.npy', allow_pickle=True)
b = np.load('./src/2_qubit_eval_x.npy', allow_pickle=True)
c = np.load('./src/2_qubit_train_y.npy', allow_pickle=True)
d = np.load('./src/2_qubit_eval_y.npy', allow_pickle=True)
e = np.load('./src/2_qubit_u.npy', allow_pickle=True)

print('\n拆分后的数据维度分别为：')
print('train_x:', a.shape, 'eval_x:', b.shape, 'train_y:', c.shape, 'eval_y:', d.shape, 'u:', e.shape)


初态数据维度为： (2000, 4)
算符数据维度为 (10, 4, 4)

末态数据维度为： (10, 2000, 4)
导出完成啦

拆分后的数据维度分别为：
train_x: (1400, 4) eval_x: (600, 4) train_y: (10, 1400, 4) eval_y: (10, 600, 4) u: (10, 4, 4)
